In [1]:
import copy
import random
import math

In [31]:
N = 6

In [3]:
def initial_board():
    ret = [[0 for j in range(N)] for i in range(N)]
    t = N // 2
    ret[t - 1][t - 1] = 1
    ret[t - 1][t] = -1
    ret[t][t - 1] = -1
    ret[t][t] = 1
    return ret

In [4]:
def show_board(board):
    print("  ", end = "")
    for i in range(N):
        print("%2d" % i, end = " ")
    print()
    for i in range(N):
        print("{} ".format(i), end = "")
        for j in range(N):
            print("%2s" % str(board[i][j]), end = " ")
        print()

show_board(initial_board())

   0  1  2  3  4  5  6  7 
0  0  0  0  0  0  0  0  0 
1  0  0  0  0  0  0  0  0 
2  0  0  0  0  0  0  0  0 
3  0  0  0  1 -1  0  0  0 
4  0  0  0 -1  1  0  0  0 
5  0  0  0  0  0  0  0  0 
6  0  0  0  0  0  0  0  0 
7  0  0  0  0  0  0  0  0 


In [8]:
def print_board(board, player, avail):
    print("It's turn for ", player)
    print()
    b = copy.deepcopy(board)
    for (x, y) in avail:
        b[x][y] = "*"
    show_board(b)

print_board(initial_board(), player = 1, avail = available_positions(initial_board(), player = 1))

It's turn for  1

   0  1  2  3  4  5  6  7 
0  0  0  0  0  0  0  0  0 
1  0  0  0  0  0  0  0  0 
2  0  0  0  0  *  0  0  0 
3  0  0  0  1 -1  *  0  0 
4  0  0  * -1  1  0  0  0 
5  0  0  0  *  0  0  0  0 
6  0  0  0  0  0  0  0  0 
7  0  0  0  0  0  0  0  0 


In [9]:
def valid_pos(x, y):
    return 0 <= x and x < N and 0 <= y and y < N

def can_put(board, player, pos):
    if board[pos[0]][pos[1]] != 0:
        return False
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if dx == 0 and dy == 0:
                continue
            x, y = pos[0] + dx, pos[1] + dy
            if not valid_pos(x, y):
                continue
            if board[x][y] == -player:
                for k in range(1, N):
                    xx, yy = pos[0] + k * dx, pos[1] + k * dy
                    if not valid_pos(xx, yy):
                        break
                    if board[xx][yy] == player:
                        return True
    return False
    
def available_positions(board, player):
    ret = []
    for i in range(N):
        for j in range(N):
            if can_put(board, player, (i, j)):
                ret.append((i, j))
    return ret

def game_over(board):
    return len(available_positions(board, -1)) == 0 \
        and len(available_positions(board, 1)) == 0

def get_winner(board):
    d = {-1:0, 1:0}
    for i in range(N):
        for j in range(N):
            if board[i][j] == 0:
                continue
            d[board[i][j]] += 1
    return -1 if d[-1] > d[1] else 1

In [10]:
def play_one_step(board, player, pos):
    board[pos[0]][pos[1]] = player
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if dx == 0 and dy == 0:
                continue
            x, y = pos[0] + dx, pos[1] + dy
            if not valid_pos(x, y):
                continue
            if board[x][y] == -player:
                for k in range(1, N):
                    xx, yy = pos[0] + k * dx, pos[1] + k * dy
                    if not valid_pos(xx, yy):
                        break
                    if board[xx][yy] == player:
                        for kk in range(1, k):
                            xxx, yyy = pos[0] + kk * dx, pos[1] + kk * dy
                            board[xxx][yyy] = player
                        break
    return board

In [5]:
available_positions(initial_board, 1)

[(1, 3), (2, 4), (3, 1), (4, 2)]

In [6]:
available_positions(initial_board, -1)

[(1, 2), (2, 1), (3, 4), (4, 3)]

In [7]:
sample_board = play_one_step(initial_board, player = 1, pos = (1, 3))
sample_board

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 0],
 [0, 0, -1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [8]:
available_positions(sample_board, -1)

[(1, 2), (1, 4), (3, 4)]

In [11]:
class Player:
    def choose_pos(self, board, player, avail):
        pass

In [12]:
class Human_Player(Player):
    def choose_pos(self, board, player, avail):
        x, y = list(map(int, (input("Please input your choice: ").split())))
        return x, y

In [13]:
class Random_Player(Player):
    def choose_pos(self, board, player, avail):
        return random.choice(avail)

In [33]:
class AI_Player(Player):
    def DFS(board, player, depth, alpha, beta):
        act = (-1, -1)
        if game_over(board) or depth == 0:
            # print("Over")
            return AI_Player.evaluate(board, player), act
        
        avail = available_positions(board, player)
        if len(avail) == 0:
            return AI_Player.DFS(board, -player, depth - 1, -beta, -alpha)
        for a in avail:
            b = copy.deepcopy(board)
            board = play_one_step(board, player, a)
            tmp = AI_Player.DFS(board, -player, depth - 1, -beta, -alpha)
            # print(tmp)
            board = b
            
            value = -tmp[0]
            if value > alpha:
                if value >= beta:
                    return beta, act
                alpha = value
                act = a

        return alpha, act
    
    def val_matrix():
        if N == 8:
            return [[90,-60,10,10,10,10,-60,90],
                    [-60,-80,5,5,5,5,-80,-60],
                    [10,5,1,1,1,1,5,10],
                    [10,5,1,1,1,1,5,10],
                    [10,5,1,1,1,1,5,10],
                    [10,5,1,1,1,1,5,10],
                    [-60,-80,5,5,5,5,-80,-60],
                    [90,-60,10,10,10,10,-60,90]]
        elif N == 6:
            return [[90,-60,10,10,-60,90],
                    [-60,-80,5,5,-80,-60],
                    [10,5,1,1,5,10],
                    [10,5,1,1,5,10],
                    [-60,-80,5,5,-80,-60],
                    [90,-60,10,10,-60,90]]
    
    def get_pos_val(board, player):
        mat = AI_Player.val_matrix()
        ret = 0
        for i in range(N):
            for j in range(N):
                ret += mat[i][j] if board[i][j] == player else 0
        return ret
    
    def get_action_evaluate(board, player):
        actions = available_positions(board, player)
        mat = AI_Player.val_matrix()
        ret = 0
        for a in actions:
            # ret += mat[a[0]][a[1]]
            ret += 1
        return ret
    
    def get_stable_stones(board, player):
        avail = available_positions(board, -player)
        b = copy.deepcopy(board)
        for a in avail:
            b = play_one_step(b, -player, a)
        ret = 0
        for i in range(N):
            for j in range(N):
                if b[i][j] == player:
                    ret += 1
        return ret
    
    def evaluate(board, player):
        v1 = AI_Player.get_pos_val(board, player) \
            - AI_Player.get_pos_val(board, -player)
        v2 = AI_Player.get_action_evaluate(board, player) \
            - AI_Player.get_action_evaluate(board, -player)
        v3 = AI_Player.get_stable_stones(board, player) \
            - AI_Player.get_stable_stones(board, -player)
        return v1 + 5 * v2 + 10 * v3

    def choose_pos(self, board, player, avail):
        alpha, act = AI_Player.DFS(board, player, 5, -99999999, 99999999)
        return act

In [15]:
def play_game(player1:Player, player2:Player):
    d = {1:player1, -1:player2}
    board = initial_board()
    player = -1
    while True:
        player = -player
        avail = available_positions(board, player)
        # print_board(board, player, avail)
        
        if game_over(board):
            winner = get_winner(board)
            # print("\nGame is over!\nThe winner is player: ", winner)
            return winner
        if len(avail) == 0:
            # print("You have to pass!\n")
            continue
        
        x, y = d[player].choose_pos(board, player, avail)
        while not (x, y) in avail:
            x, y = d[player].choose_pos(board, player, avail)
        board = play_one_step(board, player, (x, y))
        # print("\n\n")

In [76]:
b = [[-1, -1, -1, -1, -1, -1],
        [-1, -1, -1, 1, -1, -1], 
        [-1, -1, -1, 1, -1, -1], 
        [-1, -1, -1, 1, -1, -1], 
        [-1, -1, -1, 1, -1, -1], 
        [0,  -1,  0, 1,  0,  0]]
AI_Player().choose_pos(b, -1, [])

(5, 4)

In [35]:
winning = 0
for i in range(100):
    if play_game(Random_Player(), AI_Player()) == -1:
        winning += 1
    print("Game {}: win {} --- loss {}".format(i, winning, i + 1 - winning))
    print("Ratio:", winning / (i + 1))

Game 0: win 1 --- loss 0
Ratio: 1.0
Game 1: win 2 --- loss 0
Ratio: 1.0
Game 2: win 3 --- loss 0
Ratio: 1.0
Game 3: win 4 --- loss 0
Ratio: 1.0
Game 4: win 4 --- loss 1
Ratio: 0.8
Game 5: win 5 --- loss 1
Ratio: 0.8333333333333334
Game 6: win 5 --- loss 2
Ratio: 0.7142857142857143
Game 7: win 5 --- loss 3
Ratio: 0.625
Game 8: win 6 --- loss 3
Ratio: 0.6666666666666666
Game 9: win 6 --- loss 4
Ratio: 0.6
Game 10: win 7 --- loss 4
Ratio: 0.6363636363636364
Game 11: win 8 --- loss 4
Ratio: 0.6666666666666666
Game 12: win 9 --- loss 4
Ratio: 0.6923076923076923
Game 13: win 9 --- loss 5
Ratio: 0.6428571428571429
Game 14: win 10 --- loss 5
Ratio: 0.6666666666666666
Game 15: win 11 --- loss 5
Ratio: 0.6875
Game 16: win 12 --- loss 5
Ratio: 0.7058823529411765
Game 17: win 13 --- loss 5
Ratio: 0.7222222222222222
Game 18: win 14 --- loss 5
Ratio: 0.7368421052631579
Game 19: win 15 --- loss 5
Ratio: 0.75
Game 20: win 16 --- loss 5
Ratio: 0.7619047619047619
Game 21: win 17 --- loss 5
Ratio: 0.7727

KeyboardInterrupt: 

In [ ]:
class MCTS_Player(Player):
    class Node:
        def __init__(self, board, player_):
            self.C = 1.0
            self.state = board
            self.player = player_
            self.actions = available_positions(board, player_)
            
            self.son = {}
            self.num = 0
            self.win = 0
        
        def select_node(self):
            val, act = -float("INF"), None
            for a in avail:
                if a in self.son:
                    y = self.son[a]
                    v = y.win / y.num + C * math.sqrt(math.log(self.num) / (1 + y.num))
                    if v > val:
                        val, act = v, a
                else:
                    v = 0 + C * math.sqrt(math.log(self.num))
                    if v > val:
                        val, act = v, a
            return val, act
        
        def expansion_node(self):
            act = random.choice(self.actions)
            board = copy.deepcopy(self.state)
            board = play_one_step(board, -self.player, act)
            self.son[act] = Node(board, -self.player)
            return self.son[act]
        
        def rollout(self):
            board = self.state
            player = self.player
            while True:
                if game_over(board):
                    return get_winner(board)
                avail = available_positions(board, player)
                act = random.choice(avail)
                board = play_one_step(board, player, act)
                player = -player
        
        def dfs(x):
            val, act = x.select_node()
            if act in x.son:
                y = x.son[act]
                dfs(y)
            else:
                
    
    def choose_pos(self, board, player, avail):
        return -1, -1